# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
port = 27017
mongo = MongoClient(port=port)

In [3]:
# confirm that our new database was created
db_name = 'uk_food'
list_db = mongo.list_database_names()
print(list_db)
if db_name in list_db:
    print(f"Database {db_name} found.")

['GutFeel_db', 'admin', 'autosaurus', 'autosaurus_2', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']
Database uk_food found.


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [6]:
# review a document in the establishments collection
db.establishments.find_one()

{'_id': ObjectId('64e2fd1cb60896e10f93fed1'),
 'FHRSID': 254250,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'PostCode': 'CT15 6DZ',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2021-08-17T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.383298', 'latitude': '51.148133'},
 'RightToReply': '',
 'Distance': 4587.362402580997,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCoun

In [7]:
# assign the collection to a variable
establishments = db.establishments

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [15]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [16]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [17]:
def find_restaurant(restaurant_name, verbose = False):
# Find a restaurant based on its name 
# Input: restaurant name (string)
# Input: boolean, set to True to print results 
# Output: list of results
    query = {'BusinessName': restaurant_name}
    results = list(establishments.find(query))

    if verbose:
        print(f"{len(results)} document(s) found:")
        print('------------')
        pprint(results)

    return results

In [18]:
# Check that the new restaurant was inserted
restaurant_name = "Penang Flavours"
results = find_restaurant(restaurant_name, True)

1 document(s) found:
------------
[{'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'Distance': 4623.972328074718,
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': True,
  'Phone': '',
  'PostCode': 'SE18 7DY',
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('64e2fe2dafde0d235d0aee18'),
  'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
  'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}]


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [19]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID': 1, 'BusinessType': 1, '_id':0}

results = list(establishments.find(query, fields))

# Display the first three results (to check they are all the same)
pprint(results[:3])

[{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1},
 {'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1},
 {'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}]


3. Update the new restaurant with the `BusinessTypeID` you found.

In [20]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id = 1

find_query = {'BusinessName': "Penang Flavours", 'BusinessType': 'Restaurant/Cafe/Canteen'}
set_query = {'BusinessTypeId': business_type_id}

establishments.update_many(find_query, {'$set': set_query})


In [21]:
# Confirm that the new restaurant was updated
results = find_restaurant(restaurant_name, True)

1 document(s) found:
------------
[{'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeId': 1,
  'Distance': 4623.972328074718,
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': True,
  'Phone': '',
  'PostCode': 'SE18 7DY',
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('64e2fe2dafde0d235d0aee18'),
  'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
  'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}]


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [22]:
# Find how many documents have LocalAuthorityName as "Dover"
query ={'LocalAuthorityName': 'Dover'}
doc_count = establishments.count_documents(query)
print(f"Documents with location in Dover: {doc_count}")

Documents with location in Dover: 994


In [23]:
# Delete all documents where LocalAuthorityName is "Dover"
db.establishments.delete_many(query)

In [24]:
# Check if any remaining documents include Dover
query ={'LocalAuthorityName': 'Dover'}
doc_count = establishments.count_documents(query)

if doc_count == 0:
    print(f"Documents with location in Dover: {doc_count}")
    print("All documents were deleted.")

Documents with location in Dover: 0
All documents were deleted.


In [25]:
# Check that other documents remain with 'find_one'
result = establishments.find_one()
pprint(result)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64e2fd1cb60896e10f9401b4'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [26]:
# Change the data type from String to Decimal for longitude
select_document = {} # All documents
update_query = [{'$set':{"geocode.longitude":{'$toDouble': "$geocode.longitude"}}}]
establishments.update_many(select_document, update_query)

In [27]:
# Change the data type from String to Decimal for latitude
select_document = {} # All documents
update_query = [{'$set':{"geocode.latitude":{'$toDouble': "$geocode.latitude"}}}]
establishments.update_many(select_document, update_query)

In [28]:
# Check that the coordinates are now numbers
query = {}
fields = {'geocode.latitude': 1, 'geocode.longitude': 1}

result = establishments.find_one(query, fields)
pprint(result)

{'_id': ObjectId('64e2fd1cb60896e10f9401b4'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}
